## POC: Sentiment analysis on user reviews using chatgpt API

##### Input: User can upload a csv of reviews ( Beer reviews scraped from https://www.beeradvocate.com/beer/top-rated/) or based use internal survey results
##### Output: Sentiment score or label (positive/negative) as defined on the reviews
##### Transformer user: Chat GPT 3.5 16K

#### Which LLM is more accurate for sentiment analysis: VADER vs Hugging face transformer vs ChatGPT?

Review from BeerAdvocate.com: (16 oz can, canned on 11/18/22, F5670; purchased 4-pack off shelf at Shangy's... poured into a fat tulip)\n\nL: slightly-hazy medium amber-colored liquid; not a huge head on the pour, but it's tight with a frayed cotton contour... only holds a thin collar, thin misty cap; does leave a big band of lace oppo of sip spot\n\nS: very floral initial pop (rosewater), followed by some fleshy orange pulp, lemon/grapefruit zest, background hint of pine... a little honeyish malt wants to poke through... gummy orange slices... only a soft essence of alcohol... excellent overall intensity\n\nT: citrus kiss, but a rush of florality right behind it... the aftertaste is spicy, with a green bitterness that slowly swells... certainly a big malty base, with a muffin/cakey quality, but it's sodden with hop resins... vague dried pale-colored fruit (apricot, sultana) slowly creeps into the aftertaste, while the residual bitterness grows some peppery hair to it... the volume on the bitterness just keeps increasing, speakers cracking and shaking; hop artifacts emerge\n\nF: its bitter bite takes hold on the tongue, mildly intensified by the ethanol, which sets and singes (and yet cools?!) a touch near the back of the palate... its body and resins make for a silky lusciousness up front, giving way to a growing heat; soft carbonation\n\nO: why did this take so long to be released out east here(?), especially considering Furious had been here for years now; this was the first Surly beer I ever heard of c. 2011 (though Darkness and Bender weren't far behind)... I am going to try the next can in a pint, which they're website suggests is the preferred vessel... aggressive and comforting (2081)"


#### Human evaluation of the review is positve 

In [147]:
review = "(16 oz can, canned on 11/18/22, F5670; purchased 4-pack off shelf at Shangy's... poured into a fat tulip)\n\nL: slightly-hazy medium amber-colored liquid; not a huge head on the pour, but it's tight with a frayed cotton contour... only holds a thin collar, thin misty cap; does leave a big band of lace oppo of sip spot\n\nS: very floral initial pop (rosewater), followed by some fleshy orange pulp, lemon/grapefruit zest, background hint of pine... a little honeyish malt wants to poke through... gummy orange slices... only a soft essence of alcohol... excellent overall intensity\n\nT: citrus kiss, but a rush of florality right behind it... the aftertaste is spicy, with a green bitterness that slowly swells... certainly a big malty base, with a muffin/cakey quality, but it's sodden with hop resins... vague dried pale-colored fruit (apricot, sultana) slowly creeps into the aftertaste, while the residual bitterness grows some peppery hair to it... the volume on the bitterness just keeps increasing, speakers cracking and shaking; hop artifacts emerge\n\nF: its bitter bite takes hold on the tongue, mildly intensified by the ethanol, which sets and singes (and yet cools?!) a touch near the back of the palate... its body and resins make for a silky lusciousness up front, giving way to a growing heat; soft carbonation\n\nO: why did this take so long to be released out east here(?), especially considering Furious had been here for years now; this was the first Surly beer I ever heard of c. 2011 (though Darkness and Bender weren't far behind)... I am going to try the next can in a pint, which they're website suggests is the preferred vessel... aggressive and comforting (2081)"
print("Vader sentiment analysis score : ")
analyzer.polarity_scores(review)

Vader sentiment analysis score : 


{'neg': 0.117, 'neu': 0.818, 'pos': 0.064, 'compound': -0.9313}

#### Vader sentiment analysis is negative

In [ ]:
import transformers
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [148]:
print("Huggingface transoformer sentiment analysis score : ")
sentiment_pipeline(review)

Huggingface transoformer sentiment analysis score : 


[{'label': 'NEGATIVE', 'score': 0.9890099167823792}]

#### Huggingface Transformer sentiment analysis is negative

In [149]:
print("ChatGPT sentiment analysis : ")
sentiment_analysis_api(review)

ChatGPT sentiment analysis : 


'positive'

#### ChatGPT sentiment analysis is positive

### Initiating a Chatgpt 3.5 api call using SAP LLM API request

In [2]:
import json
from ipywidgets import widgets
import os
import requests

In [3]:
# Replace this with the secret key (sk.txt) provided
with open('irpa-r701-genaixl-customer-fd-sk.txt') as f:
    sk = json.load(f)
    
# Replace this with the resource group provided
resource_group = widgets.Text(
    value='e98ee3d7-45fa-43cb-8773-fb106ed18d69', # encrypted resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)
resource_group

os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

response = requests.post(
        f'{os.environ["AICORE_LLM_AUTH_URL"]}/oauth/token',
        data={"grant_type": "client_credentials"},
        auth=(os.environ['AICORE_LLM_CLIENT_ID'], os.environ['AICORE_LLM_CLIENT_SECRET']),
        timeout=8000,
)
auth_token = response.json()["access_token"]

## Copy deployment URL from AILaunchPad or Wiki page shared by Usecase team
deployment_url= "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc4baab3a2f39a69"

#### Defining Sentiment_analysis function to classify a user review (input) as positive or negative

In [107]:
# Preparing the input for inference
def sentiment_analysis_api(text):
    review=f"Classify this review as a positive or negative review in one word:{text}"
    test_input = {
        "model" : "gpt-35-turbo-16k",
        "messages" : [{ "content": review,"role": "user"}],
        "max_tokens": 800
    }
    endpoint = f"{deployment_url}/chat/completions?api-version=2023-05-15" # endpoint implemented in serving engine
    #print(endpoint)
    headers = {"Authorization": f"Bearer {auth_token}",
               'ai-resource-group': resource_group.value,
               "Content-Type": "application/json"}
    response = requests.post(endpoint, headers=headers, json=test_input)
    #converting the response to Json dictionary format in python to read the content
    data = response.json()
    return data['choices'][0]['message']['content']

#sentiment_analysis_api("I don't like my iphone")

#### Reading a csv file with customer feeback data

In [117]:
import pandas as pd
reviews_df=pd.read_csv('Feedback survey.csv')
reviews_df.rename(columns={'Please let us know your comments or further feedback.': 'Feedback'},inplace=True)

#### Calling chatgpt api on the user review column & storing response from chatgpt in another column - sentiment

In [119]:
reviews_df['sentiment']= reviews_df['Feedback'].apply(lambda review:sentiment_analysis_api(review))

In [135]:
pd.set_option('display.max_colwidth', None)
reviews_df.loc[[0,11,14],:]

,Board Area,Region,Feedback,sentiment
0,Customer Success,Latin America,"Great services, great team.",positive
11,Customer Success,North America,"The information I receive from our Portal, HRDirect, and Recruiting is difficult to consume, often takes multiple requests, not responded to timely and follow-ups or escalations are necessary to be provided the necessary information for the business. Working with my HR Business Partner is outstanding. She is thoughtful, responsive and knowledgable - a true business partner. When navigating complex organization situations and changes she is able to work with us to ensure the best outcome for our employees, the business and ultimately creating a positive impact for our Customers.",Mixed
14,Customer Success,EMEA South,"Siloed and complicated. \nTotal rewards act as if they were running the company without requisite experience, knowledge, or deep-dive understanding of unique market and individual circumstances. Lack of flexibility is a nightmare also tjj on leaders that become disenfranchised. This applies to comp, exceptions, SpiFF’s… \nPiepelining of talent is non-existent. \nHR is friendly otherwise but not a challenging and value-adding business partner.",Negative


### Sentiment analysis on beer reviews scrapped from https://www.beeradvocate.com/beer/top-rated/ using beautifulsoup

In [103]:
beer_reviews_df=pd.read_csv('Reviews.csv')
beer_reviews_df.drop('Unnamed: 0',axis=1,inplace=True)
beer_reviews_df.head()

,username,beername,user_rating,user_review
0,MonDak_Joe1953,Kentucky Brunch Brand Stout,4.71,"Tap pour at the taproom. Served chilled in a small snifter glass.\nBlack colored body. No lasting head. No lacing, but leaves a sheen.\nAroma of dark roasted malt, bourbon barrel, maple syrup, caramel, toffee, and a bit of dark fruit/cherry.\nTaste is sweet from the maple syrup and bourbon barrel, also vanilla. Flavors of fudge chocolate, caramel, toffee, and dark fruit - cherry and figs. Long lasting aftertaste with chocolate, bourbon, and toffee.\nThicker mouthfeel with syrup. Higher ABV% only detected in the vapors. Quiet carbonation.\nRich and decadent dessert beer. Treat of the day, and from the source."
1,Rristow,Kentucky Brunch Brand Stout,5.00,"The perfect barrel aged stout. Not overly sweet, nice barrel notes with coffee and maple. Just delish!"
2,BEERchitect,Kentucky Brunch Brand Stout,4.80,"The flirtation with maple comes to a crescendo in Toppling Goliath's imperial maple, coffee and bourbon barrel aged stout.\n\nKentucky Brunch Brand Stout shows that something special is up when it pours with an inky impression of black brandy equipped with sticky legging swirling around an ominously still beer. As a sultry, savory and oh-so seductive scent pulls from the onyx-black beer, the mouth waters with the scent of espresso, dark chocolate, maple, sweet bourbon and port wine. Expectedly sweet, the toasty malts bring out the flavor of maple-dipped chocolate, chocolate covered espresso bean, bourbon ball candies and whisky laced molasses.\n\nBold, savory, sweet and mouth coating on the middle palate, the boldness of the stout remains in check while the more roasty edges pull from the sweetness with espresso roast, toasty, smoky and baker's chocolate that all trend spicy, lightly bitter and balancing with a chest warming offset of bourbon and all of its caramel, vanilla, toasted oak and peppery whisky flavors in closure.\n\nObviously full, rich and lavishly malted from start to finish, the stout finishes on a drying track in an off-pastry mouthfeel of spice, light bitterness and port-like warmth. It's a masterful stout that belongs on such lofty pedestals."
3,cdinardo21,Kentucky Brunch Brand Stout,4.98,"On tap at TG for part of KBBS release day - rating 2023 version; perfect black color. This is an amazing blend richness, dessert and coffee; the barrel is perfectly complimentary and not overpowering. The sweetness is palatable rather than making your teeth hurt. The mouthfeel is thick and silky. Overall, one of the best beers I've had."
4,GreenBayBA,Kentucky Brunch Brand Stout,4.70,"Perfect black color. I didn't see much foam, other than a brown ring. The smell is a perfect blend of maple, coffee, chocolate, and bourbon. The taste is strong with maple. Almost too strong. Chocolate follows, with coffee, bourbon, and vanilla. It is very sweet. The mouthfeel is thick, smooth, lucious, and full. Very nice. Overall, this is a wonderful beer, but there is too much maple flavor for it to be one of my favorite beers."


#### Sentiment analysis using chatgpt api call

In [114]:
trimed_review=pd.DataFrame(beer_reviews_df.iloc[0:5,:])
trimed_review['sentiment'] = trimed_review['user_review'].apply(lambda review:sentiment_analysis_api(review))
trimed_review

,username,beername,user_rating,user_review,sentiment
0,MonDak_Joe1953,Kentucky Brunch Brand Stout,4.71,"Tap pour at the taproom. Served chilled in a small snifter glass.\nBlack colored body. No lasting head. No lacing, but leaves a sheen.\nAroma of dark roasted malt, bourbon barrel, maple syrup, caramel, toffee, and a bit of dark fruit/cherry.\nTaste is sweet from the maple syrup and bourbon barrel, also vanilla. Flavors of fudge chocolate, caramel, toffee, and dark fruit - cherry and figs. Long lasting aftertaste with chocolate, bourbon, and toffee.\nThicker mouthfeel with syrup. Higher ABV% only detected in the vapors. Quiet carbonation.\nRich and decadent dessert beer. Treat of the day, and from the source.",Positive
1,Rristow,Kentucky Brunch Brand Stout,5.00,"The perfect barrel aged stout. Not overly sweet, nice barrel notes with coffee and maple. Just delish!",Positive
2,BEERchitect,Kentucky Brunch Brand Stout,4.80,"The flirtation with maple comes to a crescendo in Toppling Goliath's imperial maple, coffee and bourbon barrel aged stout.\n\nKentucky Brunch Brand Stout shows that something special is up when it pours with an inky impression of black brandy equipped with sticky legging swirling around an ominously still beer. As a sultry, savory and oh-so seductive scent pulls from the onyx-black beer, the mouth waters with the scent of espresso, dark chocolate, maple, sweet bourbon and port wine. Expectedly sweet, the toasty malts bring out the flavor of maple-dipped chocolate, chocolate covered espresso bean, bourbon ball candies and whisky laced molasses.\n\nBold, savory, sweet and mouth coating on the middle palate, the boldness of the stout remains in check while the more roasty edges pull from the sweetness with espresso roast, toasty, smoky and baker's chocolate that all trend spicy, lightly bitter and balancing with a chest warming offset of bourbon and all of its caramel, vanilla, toasted oak and peppery whisky flavors in closure.\n\nObviously full, rich and lavishly malted from start to finish, the stout finishes on a drying track in an off-pastry mouthfeel of spice, light bitterness and port-like warmth. It's a masterful stout that belongs on such lofty pedestals.",positive
3,cdinardo21,Kentucky Brunch Brand Stout,4.98,"On tap at TG for part of KBBS release day - rating 2023 version; perfect black color. This is an amazing blend richness, dessert and coffee; the barrel is perfectly complimentary and not overpowering. The sweetness is palatable rather than making your teeth hurt. The mouthfeel is thick and silky. Overall, one of the best beers I've had.",Positive
4,GreenBayBA,Kentucky Brunch Brand Stout,4.70,"Perfect black color. I didn't see much foam, other than a brown ring. The smell is a perfect blend of maple, coffee, chocolate, and bourbon. The taste is strong with maple. Almost too strong. Chocolate follows, with coffee, bourbon, and vanilla. It is very sweet. The mouthfeel is thick, smooth, lucious, and full. Very nice. Overall, this is a wonderful beer, but there is too much maple flavor for it to be one of my favorite beers.",Positive
